In [94]:
from mistralai.client import MistralClient

client = MistralClient('')

In [95]:
import pandas as pd

df = pd.read_csv('data1.csv')
df.head()

,Sentence,Topic
0,Dogs are known as man's best friend.,Dogs
1,My dog loves playing fetch in the backyard,Dogs
2,Some dogs are trained to assist people with di...,Dogs
3,Labrador Retrievers are popular for their frie...,Dogs
4,Puppies require a lot of attention and care,Dogs


In [96]:
# df['Topic'].unique()

In [97]:
# topics = {'Dogs':0,'Football':1,'Cuisine':2,'Travel':3,'Technology':4,'Movies':5,'Books':6,'Music':7,'Gaming':8,'Environment':9,'Education':10,'Fashion':11}
# df['Topic'] = df['Topic'].map(topics)
# df.head()

In [98]:
def only_alphanum(text:str) -> str:
    out_text = ''
    for char in text:
        if char.isalnum() == True:
            out_text += char
    return out_text

def unique_list(list_texts:list[str]) -> list[str]:
    is_alreay_take = {}
    new_list:list[str] = []
    for text in list_texts:
        temp_text = only_alphanum(text.lower())
        if is_alreay_take.get(temp_text, False) == False:
            new_list.append(text)
            is_alreay_take[temp_text] = True
    return new_list

text = df['Sentence'].to_list()
data = unique_list(text)

In [99]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [100]:
def get_text_embedding(text):
    embeddings = client.embeddings(model = "mistral-embed", input=text).data[0].embedding
    return embeddings

# print(get_text_embedding(data[0]))

In [101]:
def get_embeddings_by_chunks(data, chunk_size):
    chunks = [data[x:x+chunk_size] for x in range(0, len(data), chunk_size)]
    embeddings_response = [client.embeddings(model="mistral-embed", input=c) for c in chunks]
    embeddings = []
    for i in range(len(embeddings_response)):
        for d in embeddings_response[i].data:
            embeddings.append(d.embedding)
    return embeddings



In [102]:
# df1 = pd.DataFrame(data, columns=['Sentence'])
# df1.head()

In [103]:
df['embedding'] = get_embeddings_by_chunks(df['Sentence'].tolist(), 1000)
df.to_csv('data_with_embeddings.csv', index=False)
df.head()

,Sentence,Topic,embedding
0,Dogs are known as man's best friend.,Dogs,"[0.0005736351013183594, -0.036590576171875, 0...."
1,My dog loves playing fetch in the backyard,Dogs,"[-0.028839111328125, 0.00765228271484375, 0.01..."
2,Some dogs are trained to assist people with di...,Dogs,"[-0.024169921875, 0.01540374755859375, 0.01120..."
3,Labrador Retrievers are popular for their frie...,Dogs,"[0.0041656494140625, -0.004886627197265625, 0...."
4,Puppies require a lot of attention and care,Dogs,"[-0.055419921875, -0.003292083740234375, 0.037..."


In [104]:
embeddings = df['embedding'].to_list()
embeddings_np = np.array(embeddings).astype('float32')
type(embeddings)

list

In [105]:
print(embeddings[0])

[0.0005736351013183594, -0.036590576171875, 0.032257080078125, -0.0119476318359375, 0.013946533203125, -0.0037555694580078125, 0.05389404296875, -0.02197265625, 0.004993438720703125, -0.0281219482421875, -0.0164794921875, 0.0252227783203125, -0.01739501953125, 0.036712646484375, -0.0499267578125, 0.02825927734375, 0.01557159423828125, 0.01073455810546875, 0.0411376953125, 0.0245361328125, -0.006053924560546875, -0.01448822021484375, -0.023834228515625, 0.0152740478515625, -0.034637451171875, 0.0004303455352783203, -0.0030841827392578125, -0.040435791015625, -0.00951385498046875, 0.024932861328125, -0.011749267578125, -0.036407470703125, -0.00789642333984375, -0.00408935546875, 0.042694091796875, -0.07049560546875, -0.01800537109375, -0.0291595458984375, 0.0221710205078125, 0.005931854248046875, -0.030670166015625, -0.041412353515625, 0.017486572265625, 0.020843505859375, 0.0029582977294921875, -0.0491943359375, -0.01139068603515625, -0.020751953125, 0.023193359375, 0.00594329833984375,

In [106]:
# Cosine Similarity 
from sklearn.metrics.pairwise import cosine_similarity

def get_cosine_similarity(data, sent):
    similarities = cosine_similarity(embeddings_np, np.array(get_text_embedding(sent)).reshape(1, -1))
    # print(len(data))
    cosine_similarities = similarities.flatten()
    max_similarities = cosine_similarities.argsort()[-6:][::-1]

    for i in max_similarities:
            print("Sentence:", data[i])
            # print("Cosine Similarity:", cosine_similarities[i])
            # print(embeddings_np[i])


    # return [(data[i], cosine_similarities[i]) for i in max_similarities]

    return [(data[i], cosine_similarities[i]) for i in max_similarities]

print(get_cosine_similarity(data, data[1]))

Sentence: My dog loves playing fetch in the backyard
Sentence: Walking my dog is a daily routine I enjoy
Sentence: Dog parks provide a great opportunity for socialization
Sentence: Training a dog requires patience and consistency
Sentence: Adopting a rescue dog can be a rewarding experience
Sentence: Some dogs are trained to assist people with disabilities
[('My dog loves playing fetch in the backyard', 0.99999944496552), ('Walking my dog is a daily routine I enjoy', 0.7948213672586134), ('Dog parks provide a great opportunity for socialization', 0.7288909804638843), ('Training a dog requires patience and consistency', 0.7202682986146004), ('Adopting a rescue dog can be a rewarding experience', 0.7121477420395963), ('Some dogs are trained to assist people with disabilities', 0.7086367125234754)]


In [107]:
import numpy as np

In [108]:
# Faiss Search
import faiss

embeddings_np = np.array(embeddings).astype('float32')
d = embeddings_np.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embeddings_np)

def faiss_search(sent, k=6):
    query_embed = get_text_embedding(sent)
    query_embed = np.array(query_embed).astype('float32').reshape(1, -1)
    D, I = index.search(query_embed, k)
    sentences = [data[i] for i in I[0]]
    return sentences


print(faiss_search(data[1]))


['My dog loves playing fetch in the backyard', 'Walking my dog is a daily routine I enjoy', 'Dog parks provide a great opportunity for socialization', 'Training a dog requires patience and consistency', 'Adopting a rescue dog can be a rewarding experience', 'Some dogs are trained to assist people with disabilities']
